# Planning-Lab Lesson 1: Search Strategies

In this first lesson we will work both on Search Strategies. 

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

In order to use the environment we need first to import the packages of OpenAI Gym. Notice that due to the structure of this repository, we need to add the parent directory to the path

In [1]:
import os, sys, time, math

module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *
import gym, envs

# Uninformed search
### Assignment 1: Breadth-First Search (BFS)

Your first assignment is to implement the BFS algorithm on SmallMaze. In particular, you should implement both *tree search* and *graph search* versions of BFS that the generic bfs will call. 

The results returned by your **BFS** must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

**For the time_cost, we consider a node checked after its generation!**

After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- BFS_TreeSearch(problem)
- BFS_GraphSearch(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for *Graph Search* and *Tree Search*:

<img src="images/tree-graph-search.png" width="600">

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *BFS* algorithm, note that it refers to the implementation of the *Graph Search Version*:

<img src="images/BFS2.png" width="600">

**Here's an implementation of the BFS algorithm in its Tree Search version:**

In [2]:
def BFS_TreeSearch(problem):
    """
    Tree Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    
    if node.state == problem.goalstate:
        return build_path(node), time_cost, space_cost
    
    frontier = NodeQueue()
    frontier.add(node)    
    
    while not frontier.is_empty():
        node = frontier.remove()
        
        for action in range(problem.action_space.n):
            
            child = Node(problem.sample(node.state, action), node)
            time_cost += 1
            
            if problem.goalstate == child.state: 
                return build_path(child), time_cost, space_cost
            
            frontier.add(child)
             
        space_cost = max(space_cost, len(frontier))  

    return build_path(node), time_cost, space_cost

**Now you can implement the same BFS algorithm in the Graph Search version:**

In [3]:
def BFS_GraphSearch(problem):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    return build_path(node), time_cost, space_cost       

**The following code calls your graph search version and the tree search version of BFS and checks the results:**

In [4]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = BFS_TreeSearch(environment)
solution_gs, time_gs, memory_gs = BFS_GraphSearch(environment)

results = CheckResult_L1A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  BFS TREE SEARCH PROBLEM  ########
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 103723
Max n° of nodes in memory: 77791

##########################################
#######  BFS Graph SEARCH PROBLEM  #######
##########################################
Your solution: []
N° of nodes explored: 1
Max n° of nodes in memory: 1

> The solution is not correct, should be: 
[(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]



## Assignment 2:  Depth-Limited Search (DLS) and Iterative Deepening depth-first Search (IDS)

Your second assignment is to implement the IDS algorithm on SmallMaze. 
In particular, you are required to implement *DLS* in the *graph search* version, *DLS* in the *tree search* version, and the final *Iterative_DLS*.

Similarly to assignment 1, the results returned by your ids must be in the following form (path, Time Cost, Space Cost) described above. After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- Recursive_DLS_TreeSearch(node, problem, limit)
- Recursive_DLS_GraphSearch(node, problem, limit, explored)
- IDS(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *Depth-Limited Search (Tree Search Version)* and *Iterative deepening depth-first search (Tree Search Version)*:

<img src="images/dls.png" width="600">

<img src="images/ids.png" width="600">

Note that Node() has a depthcost variable that represents the depth of the node in the search tree. This variable is automatically set by the Node constructor: if the root node has a depthcost of 0, its children will have a depthcost increased by 1. An example is shown below. The depthcost is useful to compute the space cost for IDS. An example is shown below:

In [5]:
start = environment.startstate
root = Node(start)  # parent = None and depthcost = 0 as default
child = Node(environment.sample(start, 0), root) # the depthcost is set automatically in the Node constructor
print("Root depthcost: {}\tChild depthcost: {}".format(root.depthcost, child.depthcost))

Root depthcost: 0	Child depthcost: 1


In [6]:
def DLS(problem, limit, RDLS_Function):
    """
    DLS
    
    Args:
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    node = Node(problem.startstate, None)
    return RDLS_Function(node, problem, limit, set())

**Now have a look at the graph search implementation of DLS and implement the tree search version of the algorithm:**

In [7]:
def Recursive_DLS_GraphSearch(node, problem, limit, explored):
    """
    Recursive DLS
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        explored: completely explored nodes
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    if problem.goalstate == node.state: # Goal state check
        return build_path(node), 1, node.depthcost
    elif limit == 0: # Limit budget check for cutoff
        return "cut_off", 1, node.depthcost
    
    explored.add(node.state)
    space_cost = node.depthcost
    time_cost = 1 
    
    cut_off_occurred = False
    for action in range(problem.action_space.n): # Look around
        child = Node(problem.sample(node.state, action), node) # Child node
        
        if child.state in explored: # We add it only if not explored
            continue
            
        result = Recursive_DLS_GraphSearch(child, problem, limit-1, explored) # Recursive call
        time_cost += result[1]
        space_cost = max(space_cost, result[2])    
        
        if result[0] == "cut_off":
            cut_off_occurred = True
        elif result[0] != "failure": # Solution found
            return result[0], time_cost, space_cost
        
    if cut_off_occurred: # Solution not found but cutoff occurred
        return "cut_off", time_cost, space_cost
    else: # No solution and no cutoff: failure
        return "failure", time_cost, space_cost

In [8]:
def Recursive_DLS_TreeSearch(node, problem, limit, explored=None):
    """
    DLS (Tree Search Version)
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    #
    # YOUR CODE HERE ...
    #
    space_cost = node.depthcost
    time_cost = 1 
    #
    # YOUR CODE HERE ...
    #
    return result[0], time_cost, space_cost
    return "cut_off", time_cost, space_cost
    return "failure", time_cost, space_cost

In [9]:
def IDS(problem, DLS_Function):
    """
    Iteartive_DLS DLS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    total_time_cost = 0
    total_space_cost = 1
    for i in zero_to_infinity():
        #
        # YOUR CODE HERE ...
        #
        return 0, 0, 0, 0 # Placeholder
    return solution_dls, total_time_cost, total_space_cost, i

The following code calls your version of IDS and checks the results:

In [10]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts, iterations_ts = IDS(environment, Recursive_DLS_TreeSearch)
solution_gs, time_gs, memory_gs, iterations_gs = IDS(environment, Recursive_DLS_GraphSearch)

results = CheckResult_L1A2([solution_ts, time_ts, memory_ts, iterations_ts], [solution_gs, time_gs, memory_gs, iterations_gs], environment)
results.check_sol_gs()
results.check_sol_ts()

##########################################
#######  IDS TREE SEARCH PROBLEM  ########
##########################################
Necessary Iterations: 9
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 138298
Max n° of nodes in memory: 9

===> Your solution is correct!

##########################################
#######  IDS GRAPH SEARCH PROBLEM  #######
##########################################
Necessary Iterations: 11
Solution: [(0, 1), (0, 0), (1, 0), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 132
Max n° of nodes in memory: 11



#### Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?

# Informed Search
We will work again on the **SmallMaze** environment.

### Priority Queue

You will need a queue ordered by priority as a queue or **PriorityQueue**. The difference between the other versions of the queue is that in **PriorityQueue**, nodes are removed from the data structure based on the current lowest value. In particular, **Node** has two useful parameters (other than those used in the previous session):

- pathcost - the path cost from the root node to the current one (defaults to 0)
- value - the value of a node. Used by PriorityQueue to order its content (defaults to 0)

#### Here is an example of usage (notice the order in which nodes are removed from the queue): 

In [11]:
import os
import sys
module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *

# Create 3 nodes for state ids 1 2 3
node_1 = Node(1) # No parent, pathcost=0, value=0
node_2 = Node(2, node_1, node_1.pathcost + 1, 10) # Child of node_1, pathcost=1, value=10
node_3 = Node(3, node_1, 100, 5)  # Child of node_1, pathcost=100, value=5

p_queue = PriorityQueue()
for n in (node_1, node_2, node_3):
    p_queue.add(n)
    print("Added: {}".format(n.state))

while not p_queue.is_empty():
    print("Removed: {}".format(p_queue.remove().state))

Added: 1
Added: 2
Added: 3
Removed: 1
Removed: 3
Removed: 2


### Uniform-Cost Search (UCS)
Before moving to informed search, it is useful to see another uninformed search algorithm: the Uniform-Cost Search (UCS). You can see the implementation in the *graph search* version in the following. The cost of performing an action is supposed to be 1 (also in the assignments).

In [12]:
def present_with_higher_cost(queue, node):
    if node.state in queue:
        if queue[node.state].value > node.value: 
            return True
    return False

In [13]:
import gym
import envs

def ucs(environment):
    """
    Uniform-cost search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        path: solution as a path
    """
    
    queue = PriorityQueue()
    queue.add(Node(environment.startstate))
    
    explored = set()
    time_cost = 1
    space_cost = 1
    
    while True:
        if queue.is_empty(): 
            return None
        
        # Retrieve node from the queue
        node = queue.remove()  
        if node.state == environment.goalstate: 
            return build_path(node), time_cost, space_cost
        
        explored.add(node.state)
        
        # Look around
        for action in range(environment.action_space.n):
            
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(environment.sample(node.state, action), node, node.pathcost + 1, node.pathcost + 1)  
            time_cost += 1
            
            if child.state not in queue and child.state not in explored:
                queue.add(child)
                
            elif present_with_higher_cost(queue, child):
                queue.replace(child)
                
        space_cost = max(space_cost, len(queue) + len(explored))


#### Let's see the results:

In [14]:
# Create and render the environment
env = gym.make("SmallMaze-v0")
env.render()
solution, time, memory = ucs(env)

CheckResult_UCS(solution, time, memory, env)

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]

##########################################
#####  UNIFORM GRAPH SEARCH PROBLEM  #####
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 61
Max n° of nodes in memory: 16


### Distance Heuristics

Informed search requires a distance heuristic to estimate the distance between a state and the goal. You already have at your disposal these functions:

- *l1_norm(p1, p2)* - Computes the L1 norm (also known as the manhattan distance) between two points specified as tuples of coordinates.
- *l2_norm(p1, p2)* - Computes the L2 norm between two points specified as tuples of coordinates.
- *chebyshev(p1, p2)* - Computes the Chebyshev distance between two points specified as tuples of coordinates. Similar to the L1 norm but diagonal moves are also considered.


**Examples:**

In [15]:
p1 = (0, 2)
p2 = (4, 0)
print("L1 norm heuristic value: {}".format(Heu.l1_norm(p1, p2)))
print("L2 norm heuristic value: {}".format(Heu.l2_norm(p1, p2)))
print("Chebyshev heuristic value: {}".format(Heu.chebyshev(p1, p2)))

L1 norm heuristic value: 6
L2 norm heuristic value: 4.47213595499958
Chebyshev heuristic value: 4


## Assignment 3: Greedy Best-First Search

The first assignment is to implement the Greedy-best-first search algorithm on **SmallMaze**. In particular, you have to implement both *greedy_tree_search* and *greedy_graph_search* versions that will be called by the generic *greedy*. Use the L1 norm as a heuristic function first, then try the others to see the differences.

The results returned by greedy must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.


### Functions to implement:
- *greedy_tree_search(environment)*
- *greedy_graph_search(environment)*

In [16]:
def greedy_tree_search(environment, timeout=10000):
    """
    Greedy-best-first Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    goalpos = environment.state_to_pos(environment.goalstate)
    queue = PriorityQueue()
    #
    # YOUR CODE HERE ...
    #
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    while True:
        if time_cost >= timeout: return ("time-out", time_cost, space_cost) # timeout check
        #
        # YOUR CODE HERE ...
        #
        return None, time_cost, space_cost

In [17]:
def greedy_graph_search(environment):
    """
    Greedy-best-first Graph search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    #
    # YOUR CODE HERE ...
    #
    explored = set()
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    return None, time_cost, space_cost

**The following function calls your implementations of greedy_tree_search and greedy_graph_search:**

In [18]:
def greedy(environment, search_type):
    """
    Greedy-best-first search
    
    Args:
        problem: OpenAI Gym environment
        search_type: type of search - greedy_tree_search or greedy_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

**The following code calls your *tree search* and *graph search* version of Greedy-best-first search and checks the results:**

In [ ]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = greedy(environment, greedy_tree_search)
solution_gs, time_gs, memory_gs = greedy(environment, greedy_graph_search)
heuristic = str(input('Which heuristic did you use? (l1_norm, l2_norm, chebyshev)\n'))

results = CheckResult_L2A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], heuristic, env)
results.check_sol_ts()
results.check_sol_gs()

## Assignment 4: A* Search
The second assignment is to implement the A search algorithm on SmallMaze. In particular, you have to implement both astar_tree_search and astar_graph_search versions that the generic astar will call. Use the L1 norm* as a heuristic function first, then try the others to see the differences.

The results returned by astar must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

Functions to implement:
- *astar_tree_search(environment)*
- *astar_graph_search(environment)*

In [ ]:
def astar_tree_search(environment):
    """
    A* Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    goalpos = environment.state_to_pos(environment.goalstate)
    queue = PriorityQueue()
    #
    # YOUR CODE HERE ...
    #    
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    return None, time_cost, space_cost

In [ ]:
def astar_graph_search(environment):
    """
    A* Graph Search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    #
    # YOUR CODE HERE ...
    #
    explored = set()
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    return None, time_cost, space_cost

**The following function calls your implementations of astar_tree_search and astar_graph_search:**

In [ ]:
def astar(environment, search_type):
    """
    A* search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - astar_tree_search or astar_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

**The following code calls your *tree search* and *graph search* version of A\* search and checks the results:**

In [ ]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = astar(environment, astar_tree_search)
solution_gs, time_gs, memory_gs = astar(environment, astar_graph_search)
heuristic = str(input('Which heuristic did you use? (l1_norm, l2_norm, chebyshev)\n'))

results = CheckResult_L2A2([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], heuristic, env)
results.check_sol_ts()
results.check_sol_gs()

### Discussion
Now that you have correctly implemented both Greedy-best-first and A* what can you say about the solutions they compute? Are there significant differences in the stats? Try to play with other heuristics as well and see if your results change.